# Submitting Jobs to Kuberay

In this demo we will go over the basics of the Ray Job Submission Client in the SDK

In [ ]:
# Import pieces from codeflare-sdk
from codeflare_sdk import Cluster, ClusterConfiguration, TokenAuthentication

### Authenticate to the cluster either using the SDK or OpenShift console login

In [ ]:
# Create authentication object for user permissions
# IF unused, SDK will automatically check for default kubeconfig, then in-cluster config

# KubeConfigFileAuthentication can also be used to specify kubeconfig path manually
# auth = TokenAuthentication(
#     token = "XXXXX",
#     server = "XXXXX",
#     skip_tls=False
# )
# auth.login()

# Paste in the oc login command from
# the OpenShift console "Copy login command" after the "!"
!oc login --token=sha256~XXXX --server=https://XXXX 

### Create Cluster

In [ ]:
# Configuration of our Ray cluster
name = "raycluster-cpu"
namespace = !cat /var/run/secrets/kubernetes.io/serviceaccount/namespace
namespace = namespace[0]

# We can use the standard codeflare image or one of the newer Ray images
image = "quay.io/project-codeflare/ray:latest-py39-cu118"
# image = "rayproject/ray-ml:2.23.0-py311-gpu"

print(name, namespace, image)

In [ ]:
!oc get localqueue

The SDK will try to find the name of your default local queue based on the annotation "kueue.x-k8s.io/default-queue": "true" unless you specify the local queue manually below


In [ ]:
cluster = Cluster(ClusterConfiguration(
    name=name,
    namespace=namespace,
    head_gpus=0,
    num_gpus=0,
    num_workers=2,
    min_cpus=0.1,
    max_cpus=4,
    min_memory=1,
    max_memory=2,
    image=image,
    write_to_file=True, # When enabled Ray Cluster yaml files are written to /HOME/.codeflare/resources 
    # local_queue="local-queue-name" # Specify the local queue manually
))

In [ ]:
# Alternatively, get a running cluster object
# from codeflare_sdk import get_cluster

# cluster = get_cluster(name, namespace=namespace)

In [ ]:
# Bring up the cluster
cluster.up()
cluster.wait_ready()

In [ ]:
cluster.details()

### Ray Job Submission

* Initialize the Cluster Job Client 
* Provide an entrypoint command directed to your job script
* Set up your [runtime environment](https://docs.ray.io/en/latest/ray-core/handling-dependencies.html#runtime-environments)

In [ ]:
# Initialize the Job Submission Client
"""
The SDK will automatically gather the dashboard address and authenticate using the Ray Job Submission Client
"""
client = cluster.job_client

See if there are any existing jobs

In [ ]:
# List all existing jobs
client.list_jobs()

#### Some Sample Runtime Environments

In [ ]:
script = "hello_ray.py"
# script = "counter.py"
runtime_env = {
    "working_dir": "./scripts/",
}


In [ ]:
# import os

# script = "s3_list.py"
# script = "s3_upload_download.py"

# runtime_env={
#     "working_dir": "scripts/",
#     "excludes": ["data/"],
#     "pip": ["boto3", "botocore"],
#     "env_vars": {
#         "AWS_ACCESS_KEY_ID": os.environ.get('AWS_ACCESS_KEY_ID'),
#         "AWS_SECRET_ACCESS_KEY": os.environ.get('AWS_SECRET_ACCESS_KEY'),
#         "AWS_S3_ENDPOINT": os.environ.get('AWS_S3_ENDPOINT'),
#         "AWS_DEFAULT_REGION": os.environ.get('AWS_DEFAULT_REGION'),
#         "AWS_S3_BUCKET": os.environ.get('AWS_S3_BUCKET')
#     },
# }

### Submit the configured job

In [ ]:
submission_id = client.submit_job(
    entrypoint=f"python {script}",
    runtime_env=runtime_env,
)

print(submission_id)

### Query Important Job Information

In [ ]:
# Get the job's status
print(client.get_job_status(submission_id), "\n")

# Get job related info
print(client.get_job_info(submission_id), "\n")

# Get the job's logs
print(client.get_job_logs(submission_id))

In [ ]:
# Iterate through the logs of a job 
async for lines in client.tail_job_logs(submission_id):
    print(lines, end="")

### Delete a job

In [ ]:
print(client.list_jobs())

In [ ]:
client.stop_job(submission_id)

client.delete_job(submission_id)

In [ ]:
print(client.list_jobs())

In [ ]:
cluster.down()